In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
path='/content/drive/MyDrive/whisky_details.csv'
whisky_detail_df = pd.read_csv(path)
whisky_detail_df

,Whiskybase ID,Whiskey Link:,Category,Distillery,Bottler,Vintage,Bottled,Stated Age,Casktype,Number of bottles,Strength,Size,Bottled for,Market,Price,Overall rating,Number of votes
0,WB221509,https://www.whiskybase.com/whiskies/whisky/221...,Blended Malt,NaN,Chivas Brothers Ltd. (Chvs),NaN,19.04.2022,08 years old,NaN,NaN,40.0 % Vol.,700 ml,NaN,NaN,€ 11.50,74.00,1
1,WB226993,https://www.whiskybase.com/whiskies/whisky/226...,Blended Malt,NaN,Chivas Brothers Ltd. (Chvs),NaN,14.05.2022,08 years old,NaN,NaN,42.8 % Vol.,750 ml,NaN,Thailand,USD$ 20.00,83.50,2
2,WB218272,https://www.whiskybase.com/whiskies/whisky/218...,Single Malt,The Glasgow Distillery Co.,Distillery Bottling,13.07.2015,22.07.2022,NaN,Ruby Port,388.0,62.5 % Vol.,500 ml,Kirsch Import Germany Exclusive,Germany,€ 84.74,86.06,20
3,WB220758,https://www.whiskybase.com/whiskies/whisky/220...,Single Malt,The Glasgow Distillery Co.,Distillery Bottling,12.05.2016,25.08.2022,NaN,1st Fill Ex-Bourbon / Tokaji Barrique Finish,274.0,59.3 % Vol.,700 ml,Kirsch Import,Germany,€ 89.90,85.60,25
4,WB205065,https://www.whiskybase.com/whiskies/whisky/205...,Single Malt,The Glasgow Distillery Co.,Distillery Bottling,23.08.2017,08.02.2022,04 years old,1st fill Marsala,408.0,55.8 % Vol.,500 ml,Glasgow Whisky Festival Exclusive,NaN,£ 140.55,85.14,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6628,WB208338,https://www.whiskybase.com/whiskies/whisky/208...,Single Malt,Yserrain,Distillery Bottling,NaN,2022,NaN,ex-Pedro Ximènez Sherry,NaN,44.0 % Vol.,500 ml,NaN,Germany,€ 54.90,75.00,1
6629,WB208340,https://www.whiskybase.com/whiskies/whisky/208...,Single Malt,Yserrain,Distillery Bottling,NaN,2022,NaN,NaN,NaN,45.0 % Vol.,500 ml,NaN,Germany,€ 54.90,75.00,1
6630,WB199115,https://www.whiskybase.com/whiskies/whisky/199...,Single Malt,Yuza Distillery,Distillery Bottling,2018,2022,NaN,NaN,8500.0,61.0 % Vol.,700 ml,NaN,NaN,€ 302.05,85.41,24
6631,WB226284,https://www.whiskybase.com/whiskies/whisky/226...,Single Malt,Yuza Distillery,Distillery Bottling,2019,2022,NaN,NaN,NaN,62.0 % Vol.,700 ml,NaN,Japan,￥ 37164.76,84.33,5


In [ ]:
whisky_detail_df['Category'].unique()

array(['Blended Malt', 'Single Malt', 'Bourbon', 'Corn', 'Blended Grain',
       'Rye', 'Single Pot Still', 'Single Grain', 'Blend', 'Spirit',
       'American Whiskey', nan, 'Tennessee', 'Wheat', 'Canadian Whisky'],
      dtype=object)

In [ ]:
whisky_detail_df['Stated Age'].unique()

array(['08 years old', nan, '04 years old', '13 years old',
       '21 years old', '37 years old', '06 years old', '03 years old',
       '07 years old', '15 years old', '31 years old', '30 years old',
       '14 years old', '12 years old', '17 years old', '16 years old',
       '24 years old', '18 years old', '19 years old', '20 years old',
       '10 years old', '11 years old', '32 years old', '09 years old',
       '25 years old', '26 years old', '22 years old', '05 years old',
       '23 years old', '52 years old', '28 years old', '33 years old',
       '27 years old', '39 years old', '40 years old', '45 years old',
       '34 years old', '50 years old', '48 years old', '42 years old',
       '38 years old', '02 years old', '29 years old', '44 years old',
       '35 years old', '46 years old', '01 year old', '74 years old',
       '70 years old', '67 years old', '63 years old', '58 years old',
       '53 years old', '36 years old', '56 years old', '49 years old',
       '47 years o

In [ ]:
whisky_detail_df['Casktype'].unique()

array([nan, 'Ruby Port', '1st Fill Ex-Bourbon / Tokaji Barrique Finish',
       ..., '1st fill rum', 'ex-Pedro Ximènez Sherry', 'Ærø Oak'],
      dtype=object)

In [ ]:
whisky_detail_df['Vintage'] = pd.to_datetime(whisky_detail_df['Vintage'], errors='coerce').dt.year.fillna(0).astype(int)
whisky_detail_df['Bottled'] = pd.to_datetime(whisky_detail_df['Bottled'], errors='coerce').dt.year.fillna(0).astype(int)
whisky_detail_df['Stated Age'] = whisky_detail_df['Stated Age'].str.extract('(\d+)').fillna(0).astype(int)
whisky_detail_df['Strength'] = whisky_detail_df['Strength'].str.extract('(\d+\.\d+)')
whisky_detail_df[['Currency', 'Price']] = whisky_detail_df['Price'].str.split(' ', 1, expand=True)

<ipython-input-6-71fe0c4f2a21>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  whisky_detail_df['Vintage'] = pd.to_datetime(whisky_detail_df['Vintage'], errors='coerce').dt.year.fillna(0).astype(int)
<ipython-input-6-71fe0c4f2a21>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  whisky_detail_df['Bottled'] = pd.to_datetime(whisky_detail_df['Bottled'], errors='coerce').dt.year.fillna(0).astype(int)
<ipython-input-6-71fe0c4f2a21>:5: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  whisky_detail_df[['Currency', 'Price']] = whisky_detail_df['Price'].str.split(' ', 1, expand=True)


In [ ]:
whisky_detail_df.loc[(whisky_detail_df['Vintage'] == 0) & (whisky_detail_df['Bottled'] != 0) & (whisky_detail_df['Stated Age'] != 0), 'Vintage'] = whisky_detail_df['Bottled'] - whisky_detail_df['Stated Age']
whisky_detail_df.loc[(whisky_detail_df['Bottled'] == 0) & (whisky_detail_df['Vintage'] != 0) & (whisky_detail_df['Stated Age'] != 0), 'Bottled'] = whisky_detail_df['Vintage'] + whisky_detail_df['Stated Age']
whisky_detail_df.loc[(whisky_detail_df['Stated Age'] == 0) & (whisky_detail_df['Vintage'] != 0) & (whisky_detail_df['Bottled'] != 0), 'Stated Age'] = whisky_detail_df['Bottled'] - whisky_detail_df['Vintage']

In [ ]:
size_pattern = r'(\d+)\s*(ml|ML|cl|CL|L|l)'
whisky_detail_df[['Size', 'Size_Unit']] = whisky_detail_df['Size'].str.extract(size_pattern,0, expand=True)
whisky_detail_df.drop(["Size_Unit"], axis=1)

,Whiskybase ID,Whiskey Link:,Category,Distillery,Bottler,Vintage,Bottled,Stated Age,Casktype,Number of bottles,Strength,Size,Bottled for,Market,Price,Overall rating,Number of votes,Currency
0,WB221509,https://www.whiskybase.com/whiskies/whisky/221...,Blended Malt,NaN,Chivas Brothers Ltd. (Chvs),2014,2022,8,NaN,NaN,40.0,700,NaN,NaN,11.50,74.00,1,€
1,WB226993,https://www.whiskybase.com/whiskies/whisky/226...,Blended Malt,NaN,Chivas Brothers Ltd. (Chvs),2014,2022,8,NaN,NaN,42.8,750,NaN,Thailand,20.00,83.50,2,USD$
2,WB218272,https://www.whiskybase.com/whiskies/whisky/218...,Single Malt,The Glasgow Distillery Co.,Distillery Bottling,2015,2022,7,Ruby Port,388.0,62.5,500,Kirsch Import Germany Exclusive,Germany,84.74,86.06,20,€
3,WB220758,https://www.whiskybase.com/whiskies/whisky/220...,Single Malt,The Glasgow Distillery Co.,Distillery Bottling,2016,2022,6,1st Fill Ex-Bourbon / Tokaji Barrique Finish,274.0,59.3,700,Kirsch Import,Germany,89.90,85.60,25,€
4,WB205065,https://www.whiskybase.com/whiskies/whisky/205...,Single Malt,The Glasgow Distillery Co.,Distillery Bottling,2017,2022,4,1st fill Marsala,408.0,55.8,500,Glasgow Whisky Festival Exclusive,NaN,140.55,85.14,9,£
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6628,WB208338,https://www.whiskybase.com/whiskies/whisky/208...,Single Malt,Yserrain,Distillery Bottling,0,2022,0,ex-Pedro Ximènez Sherry,NaN,44.0,500,NaN,Germany,54.90,75.00,1,€
6629,WB208340,https://www.whiskybase.com/whiskies/whisky/208...,Single Malt,Yserrain,Distillery Bottling,0,2022,0,NaN,NaN,45.0,500,NaN,Germany,54.90,75.00,1,€
6630,WB199115,https://www.whiskybase.com/whiskies/whisky/199...,Single Malt,Yuza Distillery,Distillery Bottling,2018,2022,4,NaN,8500.0,61.0,700,NaN,NaN,302.05,85.41,24,€
6631,WB226284,https://www.whiskybase.com/whiskies/whisky/226...,Single Malt,Yuza Distillery,Distillery Bottling,2019,2022,3,NaN,NaN,62.0,700,NaN,Japan,37164.76,84.33,5,￥


In [ ]:
bottles_data = whisky_detail_df[['Number of bottles']]
knn_imputer = KNNImputer(n_neighbors=5)
whisky_detail_df["Number of bottles"] = knn_imputer.fit_transform(bottles_data).round()

In [ ]:
whisky_detail_df["Number of bottles"] = whisky_detail_df["Number of bottles"].astype(int)
whisky_detail_df["Strength"] = whisky_detail_df["Strength"].astype(float)
whisky_detail_df["Price"] = whisky_detail_df["Price"].astype(float)

In [ ]:
whisky_detail_df["Category"].fillna("Other", inplace=True)
whisky_detail_df["Distillery"].fillna("Other", inplace=True)
whisky_detail_df["Casktype"].fillna("Other", inplace=True)
whisky_detail_df["Market"].fillna("Other", inplace=True)
whisky_detail_df["Currency"].fillna("Other", inplace=True)

In [ ]:
pip install forex-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
from forex_python.converter import CurrencyRates
cache={}
def convert_to_usd(row):
    currency = row['Currency']
    price = row['Price']
    if currency == 'USD$':
        return price
    else:
        currency_names = {
            '€': 'EUR',
            '£': 'GBP',
            'CHF': 'CHF',
            'AUD$': 'AUD',
            'SEK': 'SEK',
            'C$': 'CAD',
            '￥': 'JPY',
            'NOK': 'NOK'
        }
        if currency in currency_names:
            if currency in cache:
              exchange_rate = cache[currency]
            else:
              c = CurrencyRates()
              exchange_rate = c.get_rate(currency_names[currency], 'USD')
              cache[currency] =exchange_rate
            return round(price * exchange_rate,2)
        else:
          return "Other"


In [ ]:
whisky_detail_df['Price'] = whisky_detail_df.apply(convert_to_usd, axis=1)

In [ ]:
whisky_detail_df

,Whiskybase ID,Whiskey Link:,Category,Distillery,Bottler,Vintage,Bottled,Stated Age,Casktype,Number of bottles,Strength,Size,Bottled for,Market,Price,Overall rating,Number of votes,Currency,Size_Unit
0,WB221509,https://www.whiskybase.com/whiskies/whisky/221...,Blended Malt,Other,Chivas Brothers Ltd. (Chvs),2014,2022,8,Other,710,40.0,700,NaN,Other,12.4,74.00,1,€,ml
1,WB226993,https://www.whiskybase.com/whiskies/whisky/226...,Blended Malt,Other,Chivas Brothers Ltd. (Chvs),2014,2022,8,Other,710,42.8,750,NaN,Thailand,20.0,83.50,2,USD$,ml
2,WB218272,https://www.whiskybase.com/whiskies/whisky/218...,Single Malt,The Glasgow Distillery Co.,Distillery Bottling,2015,2022,7,Ruby Port,388,62.5,500,Kirsch Import Germany Exclusive,Germany,91.35,86.06,20,€,ml
3,WB220758,https://www.whiskybase.com/whiskies/whisky/220...,Single Malt,The Glasgow Distillery Co.,Distillery Bottling,2016,2022,6,1st Fill Ex-Bourbon / Tokaji Barrique Finish,274,59.3,700,Kirsch Import,Germany,96.91,85.60,25,€,ml
4,WB205065,https://www.whiskybase.com/whiskies/whisky/205...,Single Malt,The Glasgow Distillery Co.,Distillery Bottling,2017,2022,4,1st fill Marsala,408,55.8,500,Glasgow Whisky Festival Exclusive,Other,176.6,85.14,9,£,ml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6628,WB208338,https://www.whiskybase.com/whiskies/whisky/208...,Single Malt,Yserrain,Distillery Bottling,0,2022,0,ex-Pedro Ximènez Sherry,710,44.0,500,NaN,Germany,59.18,75.00,1,€,ml
6629,WB208340,https://www.whiskybase.com/whiskies/whisky/208...,Single Malt,Yserrain,Distillery Bottling,0,2022,0,Other,710,45.0,500,NaN,Germany,59.18,75.00,1,€,ml
6630,WB199115,https://www.whiskybase.com/whiskies/whisky/199...,Single Malt,Yuza Distillery,Distillery Bottling,2018,2022,4,Other,8500,61.0,700,NaN,Other,325.61,85.41,24,€,ml
6631,WB226284,https://www.whiskybase.com/whiskies/whisky/226...,Single Malt,Yuza Distillery,Distillery Bottling,2019,2022,3,Other,710,62.0,700,NaN,Japan,266.66,84.33,5,￥,ml


In [ ]:
features = whisky_detail_df[["Category","Stated Age","Strength","Market","Price","Overall rating","Number of votes"]]

In [ ]:
features["Price"] = pd.to_numeric(features["Price"], errors='coerce')

<ipython-input-18-4644c4526cca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features["Price"] = pd.to_numeric(features["Price"], errors='coerce')


In [ ]:
for each in features.columns:
    print(each, features[each].nunique())
    print(features[each].isnull().sum())

Category 15
0
Stated Age 64
0
Strength 338
16
Market 49
0
Price 3798
904
Overall rating 869
0
Number of votes 174
0


In [ ]:
features[features['Stated Age'] == 0]

,Category,Stated Age,Strength,Market,Price,Overall rating,Number of votes
22,Single Malt,0,46.0,Other,58.11,82.89,55
24,Single Malt,0,46.0,Other,52.00,82.82,35
25,Single Malt,0,46.0,Other,78.72,68.00,3
67,Single Malt,0,46.0,Other,31.41,79.00,1
106,Single Malt,0,61.2,Other,79.98,85.15,36
...,...,...,...,...,...,...,...
6622,Bourbon,0,50.5,Other,116.99,87.50,2
6625,Single Malt,0,48.0,Other,267.21,86.00,22
6626,Single Malt,0,48.0,Other,405.58,87.33,11
6628,Single Malt,0,44.0,Germany,59.18,75.00,1


In [ ]:
features[features['Strength'] == 0]

,Category,Stated Age,Strength,Market,Price,Overall rating,Number of votes


In [ ]:
features[features['Price'].isna()]

,Category,Stated Age,Strength,Market,Price,Overall rating,Number of votes
27,Single Malt,21,48.30,Germany,NaN,90.18,13
35,Single Malt,15,53.50,Other,NaN,86.00,1
36,Single Malt,31,52.60,United Kingdom,NaN,90.25,12
38,Single Malt,31,51.20,Other,NaN,89.50,5
39,Single Malt,30,49.60,Other,NaN,89.50,2
...,...,...,...,...,...,...,...
6595,Bourbon,5,58.23,Other,NaN,88.00,2
6607,Bourbon,5,52.50,Other,NaN,85.75,4
6615,Single Malt,5,62.00,Japan,NaN,83.00,1
6623,Single Malt,3,67.50,Israel,NaN,79.00,1


In [ ]:
features = features[features['Stated Age'] != 0]
features = features.dropna(subset=['Price',"Strength"])
features = features[features['Strength'] != 0]

In [ ]:
features[:20]

,Category,Stated Age,Strength,Market,Price,Overall rating,Number of votes
0,Blended Malt,8,40.0,Other,12.40,74.00,1
1,Blended Malt,8,42.8,Thailand,20.00,83.50,2
2,Single Malt,7,62.5,Germany,91.35,86.06,20
3,Single Malt,6,59.3,Germany,96.91,85.60,25
4,Single Malt,4,55.8,Other,176.60,85.14,9
5,Single Malt,4,56.0,Other,83.61,84.89,14
6,Single Malt,4,55.0,Other,70.34,87.00,2
7,Single Malt,4,58.1,Other,74.09,85.00,1
8,Single Malt,4,57.5,Other,88.28,85.50,13
9,Single Malt,4,60.5,Germany,90.42,84.00,14


In [ ]:
features.isnull().sum()

Category           0
Stated Age         0
Strength           0
Market             0
Price              0
Overall rating     0
Number of votes    0
dtype: int64

In [ ]:
categorical_columns = ['Category']
numerical_columns = []

In [ ]:
# pipeline_categorical = Pipeline(
#     [
#         ("onehot", OneHotEncoder()),
#     ]
# )

# pipeline_numerical = Pipeline(
#     [
#         ("scaler", StandardScaler()),
#     ]
# )

# pipeline_full = ColumnTransformer(
#     [
#         ("categorical", pipeline_categorical, categorical_columns),
#         ("numerical", pipeline_numerical, numerical_columns),
#     ]
# )

In [ ]:
# pipeline_full.fit(features)

In [ ]:
# encoded_features = pipeline_full.transform(features)

In [ ]:
# encoded_features = pd.DataFrame(features, columns=pipeline_full.get_feature_names_out(input_features=features.columns))

In [ ]:
# encoded_features

In [ ]:
encoded_features = pd.get_dummies(features, columns=['Category'], prefix='', prefix_sep='')
encoded_features = pd.get_dummies(encoded_features, columns=['Market'], prefix='', prefix_sep='')

In [ ]:
encoded_features

,Stated Age,Strength,Price,Overall rating,Number of votes,American Whiskey,Blend,Blended Grain,Blended Malt,Bourbon,...,Singapore,Slovakia,Spain,Sweden,Switzerland,Taiwan,Thailand,Ukraine,United Kingdom,United States
0,8,40.0,12.40,74.00,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,8,42.8,20.00,83.50,2,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,7,62.5,91.35,86.06,20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,59.3,96.91,85.60,25,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,55.8,176.60,85.14,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6621,6,50.0,76.56,84.36,13,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6624,10,45.0,242.80,87.04,31,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6630,4,61.0,325.61,85.41,24,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6631,3,62.0,266.66,84.33,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
